In [ ]:
from collections import deque

import numpy as np
%config Completer.use_jedi = False

In [ ]:
import supersuit as ss
from pettingzoo.mpe import simple_adversary_v2

# Creating simple_adversary_v2 parallel env
orig_env = simple_adversary_v2.parallel_env()
action_spaces = orig_env.action_spaces
env = ss.pad_observations_v0(orig_env)
env = ss.pettingzoo_env_to_vec_env_v0(env)

num_envs = 2
env = ss.concat_vec_envs_v0(env, num_envs, num_cpus=4, base_class="gym")

In [ ]:
env.reset()

In [ ]:
observations, rewards, dones, infos = env.step([env.action_space.sample() for i in range(env.num_envs)])
# TODO: implement training loop
# TODO: implement model with just one policy per agent
# TODO: implement policy ensemble

In [ ]:
num_agents = orig_env.max_num_agents
state_size = env.observation_space.shape[0]
action_space_size = env.action_space.n

print("Num of agents: ", num_agents)
print("State Size: ", state_size)
print("Action Space Size: ", action_space_size)

In [ ]:
from agent import MultiAgentActorCritic
agent = MultiAgentActorCritic(num_agents, num_envs, state_size, action_space_size)

In [ ]:
n_episodes = 100
train_mode = True
max_t = 26

scores_window = deque(maxlen=100)  # last 100 scores
scores = []

for i_episode in range(1, n_episodes + 1):
    states = env.reset()

    score = np.zeros((num_agents * num_envs, ))
    for t in range(max_t):
        actions = agent.act(states)
        try:
            next_states, rewards, dones, infos = env.step(actions)
        except Exception as e:
            print(e)
            break

        if train_mode:
            agent.step(states, actions, rewards, next_states, dones)
        states = next_states
        score += rewards
        if np.any(dones):
            break

    score = score.reshape((num_envs, num_agents)).mean(axis=0)
    scores_window.append(score)  # save most recent score
    scores.append(score)  # save most recent score
    mean_score_window = np.mean(scores_window, axis=0)
    print(
        f"\rEpisode {i_episode}\tAverage Score: {mean_score_window}",
        end="",
    )
    if i_episode % 5 == 0:
        print(f"\rEpisode {i_episode}\tAverage Score: {mean_score_window}")